## Logging System

Logging is useful for different kind of people:

* Core developpers: *while* developping his software, a programmer places logging instructions at some places in order to understand what is happening;
* Functionnal test writers: when writing a test, a tester tries to understand the logs produced by the software;
* End-users: messages are sometimes targetted to an end-user;
* Maintenance engieneers: the whole set of messages produced by the software is often used by a maintenance engineer to understand a problem in a software and to repair it when possible.

According to this list, it must be clear that logs are written from day-0 of a software, and might be used in production or even in post-mortem diagnostic.

All modern languages provide a powerful logging API that ease high level programming:

* Java has its own [java.logging API](https://docs.oracle.com/javase/7/docs/api/java/util/logging/package-summary.html), and also some alternatives such as [`log4j`](http://logging.apache.org/log4j/2.x/), or [`logback`](http://logback.qos.ch/);
* Python includes the [logging](https://docs.python.org/2/library/logging.html) module;
* Go provides the [log](https://golang.org/pkg/log/) package.
* ...

However C provides nothing. POSIX provides `syslog()` but as its name implies it has been designed for the system: messages sent by a sofware, are received by the system loggers that decides what to do with the message at a system level. Therefore, even if the end-user can change the logging level of its software, it cannot change anything else: in particular, it usually has not the permission to read the file where the logs are written to (e.g.: `/var/log/message`).

To achieve high level programming, the logging system must be used for any message oriented output whether being on screen, on a file, or over the network. This has several advantages compared to the use of a logging API in addition to the standard `printf()` for console output:

* development simplification (one API to learn only, but you are right, any C programmer knows `printf()`);
* performance optimization: a single call might produce many outputs instead of one call with the logging API and one call for the console output with `printf()`;
* customizability: the software uses the logging API to produce messages, but how and where those messages are produced 
  is defined by the configuration.   

As an example, with a correct configuration one can guarantee that all messages produced on the console (and seen by the end users) are also recorded to a file (that can be hidden if required). The file might contain much more information (according to the configured verbosity), but always contain the messages that have been seen by the end-user. This can be quite usefull for debugging and/or maintenance.

Therefore, we propose the `bxilog.h` module.  

### The `bxilog`  API

When writing a software, the programmer must focus on what messages must be produced and at what level. 
Where messages will be routed comes in second at the configuration phase. Note that in the case of a library software, this last configuration step is entirely skipped. It will be the responsability of the `main()` program to configure the logging framework.

The example below presents a basic use of the API:

In [20]:
#include <errno.h>
#include <bxi/base/err.h>
#include <bxi/base/log.h>
#include <bxi/base/log/console_handler.h> 

// (1) Define static loggers
SET_LOGGER(MAIN_LOGGER, "ex.main");
SET_LOGGER(ANOTHER_LOGGER, "ex.other");
 
void output_level_names() {
    TRACE(ANOTHER_LOGGER, "Fetching level names"); // (2) Use loggers
    
    char ** level_names;
    size_t n = bxilog_level_names(&level_names);
    
    OUT(MAIN_LOGGER, "Found %zu level names", n);  // (3) Level OUT replaces printf() like statement
    
    for (size_t i = 0; i < n; i++) {
        DEBUG(MAIN_LOGGER, "Level %zu:\t %s", i, level_names[i]);
    }
}

void output_logger_names() {
    TRACE(ANOTHER_LOGGER, "Fetching logger names")
        
    bxilog_logger_p * loggers;
    size_t n = bxilog_registry_getall(&loggers);   
    OUT(MAIN_LOGGER, "Currently, %zu loggers are known:", n);
    
    for (size_t i = 0; i < n; i++) {
        DEBUG(MAIN_LOGGER, "%s", loggers[i]->name);
    }
        
}
 
int main() {
    // (4) Create a new empty configuration 
    bxilog_config_p config = bxilog_config_new("API Example"); // You might prefer to use argv[0]
    
    bxilog_filters_p filters;
    // (5) Define filters: comment next line, and uncomment the second line to see filtering in action
    bxierr_p err = bxilog_filters_parse(":lowest", &filters); // Show everything
    // bxierr_p err = bxilog_filters_parse(":output,ex.main:debug,~bxilog:off", &filters);
    bxierr_abort_ifko(err);
    
    
    // (6) Add a console handler to the configuration
    bxilog_config_add_handler(config, 
                              BXILOG_CONSOLE_HANDLER,
                              filters, 
                              BXILOG_WARNING,
                              12, // Use 12 characters maximum when displaying logger name
                              BXILOG_COLORS_NONE);
    
    // (7) Initialize the bxilog library
    err = bxilog_init(config);
    bxierr_abort_ifko(err);
    
    // From now, the logging library can be used as a full replacement for 
    // fprintf() and the like.
    INFO(MAIN_LOGGER, "Starting the program")
    
    output_level_names();
    
    // (8) You might also use dynamic loggers if required
    bxilog_logger_p logger;
    err = bxilog_registry_get("ex.dynamic", &logger);
    bxierr_abort_ifko(err);
    
    WARNING(logger, "Producing a log through a dynamic logger");
    
    
    output_logger_names();   
    
    // (10) Illustrate error reporting
    char * filename = "/tmp/non_existing_file_foo_bar";
    errno = 0;
    int rc = open(filename, O_RDONLY);
    BXIASSERT(MAIN_LOGGER, -1 == rc);
    err = bxierr_errno("An error happened while opening file: %s", filename);
    BXILOG_REPORT(MAIN_LOGGER, BXILOG_ERROR, err, "Non critical error -- just for testing");
    
    // (11) Finalize the bxilog library
    err = bxilog_finalize(true);
    bxierr_abort_ifko(err);
}

[W] ex.dynamic   Producing a log through a dynamic logger
[E] ex.main      Non critical error -- just for testing
[E] ex.main      ##code## 2
[E] ex.main      ##mesg## An error happened while opening file: /tmp/non_existing_file_foo_bar: No such file or directory


[D] ~bxilog      Initialization done
[I] ex.main      Starting the program
[T] ex.other     Fetching level names
Found 13 level names
[D] ex.main      Level 0:	 off
[D] ex.main      Level 1:	 panic
[D] ex.main      Level 2:	 alert
[D] ex.main      Level 3:	 critical
[D] ex.main      Level 4:	 error
[D] ex.main      Level 5:	 warning
[D] ex.main      Level 6:	 notice
[D] ex.main      Level 7:	 output
[D] ex.main      Level 8:	 info
[D] ex.main      Level 9:	 debug
[D] ex.main      Level 10:	 fine
[D] ex.main      Level 11:	 trace
[D] ex.main      Level 12:	 lowest
[T] ex.other     Fetching logger names
Currently, 11 loggers are known:
[D] ex.main      ~bxilog.logger
[D] ex.main      ~bxilog
[D] ex.main      ~bxilog.cfg
[D] ex.main      ~bxilog.lvl
[D] ex.main      ~bxilog.signal
[D] ex.main      ~bxilog.fork
[D] ex.main      ~bxilog.netsnmp
[D] ex.main      bxilog.remote
[D] ex.main      ex.other
[D] ex.main      ex.main
[D] ex.main      ex.dynamic
[T] ex.main      ##trce## Backtrace of

#### Loggers

In order to produce a log, a thread needs a logger. A logger is basically defined by an arbitrary name. The convention is to create a pseudo tree hierarchy using the '.' as a character separator for different loggers. This is important for filtering. More on that later.

Loggers can be defined statically as in (1) or dynamically as in (8). The only difference is that static loggers are created at loading time (when the library is loaded in memory) whereas dynamic loggers are created the first time the `bxilog_registry_get()` function is called with a logger name unknown to the system. The set of loggers currently known by the system can be retrieved using `bxilog_registry_getall()`.

Once you have a logger, producing a log can be done using various macros which name represents the logging level as shown in (2) with the level `TRACE`.

#### Logging levels

The bxilog module defines several logging levels that is a superset of syslog ones. Retrieving the set of logging levels can be done using the `bxilog_get_all_level_names()`.

The `OUTPUT` level should be used as a replacement for `printf()` as shown in (3). Notice also that levels from `NOTICE` up to `PANIC` should also be used as a replacement for `fprintf(stderr)`. 

How and where those logs will be produced is given by the configuration.

#### Configuration of log handlers

In order to initialize the bxilog library (7), a configuration must be created as in (4) and a logging handler should be added to it as in (6). Several handlers can be added sequentially. A logging handler consumes logs produced by threads and normally outputs them somewhere: on the console, on the filesystem, over the network, and so on. 

For example, the `BXILOG_CONSOLE_HANDLER` displays logs with level 'LOWEST' up to 'NOTICE' on the process standard output file descriptor (stdout) but logs with level 'WARNING' up to 'PANIC' on the process standard error file descriptor (stderr). It also supports colorization of log according to their level. Other logging handler might behave differently.

A simple API is provided to implement new logging handlers. Currently, bxilog provides:

* `BXILOG_CONSOLE_HANDLER`: produces logs on either stdout or stderr according to a customizable log    
  level with or without customizable colors;
* `BXILOG_FILE_HANDLER`: produces logs in a file with all details;
* `BXILOG_SYSLOG_HANDLER`: sends the logs to the syslog daemon;
* `BXILOG_NETSNMP_HANDLER`: produces logs to the net-snmp logging library; 
* `BXILOG_REMOTE_HANDLER`: sends the logs to another process;


In the example above, two logs have been output but they have not been produced by our code. Those logs have a logging name starting with `~bxilog`: they are produced by the logging library itself. This is a perfect example of libraries written with the logging system: it might help in the debugging process. However, most of the time, those logs must not be seen. This is where filtering takes place. 

#### Filtering

Filtering occurs at two different places: first, each logger is given a logging level. Therefore, when a thread is requesting the production of a log at the TRACE level as in (2), if the logger is currently having level 'OUTPUT', the log won't be produced at all. Second, each log handler filters each received log according to the logger name used for the production of the log, and the level of that log.

Filtering is expressed by a simple string, and processed with the `bxilog_filters_parse()` function as in (6). As an example, change (5) by commenting out the first line and uncommenting the second. The output will be rather different.

This is a very simple yet very efficient mechanism to customize the logging system. 

### Advanced  Features

The previous section presented the main API of the `bxilog` module. However, the `bxilog` module provides many other advanced features presented in this section.  

#### Error Reporting

We have seen that `bxierr` is a good error mechanism for high level programming. However, if no error reporting system is provided, it reduces significantly its purpose. Therefore `bxilog` supports error reporting as shown in (10). An error report is produced using the `BXILOG_REPORT` macro which destroys the error. This report includes many information such as the error code, the logging message and the backtrace. Note that the actual error message and the backtrace are not produced at the same logging level. This allows the console output to display a simple error message while the filesystem can contain a file with both the error message and the backtrace thanks to well defined filters.

`Bxilog` also provides `BXIASSERT()` as a replacement for the standard `assert()` function. The main advantage is that if you use `assert()`, pending logs might not be flushed by the various handlers and might get lost. Also, `BXIASSERT()` provides more information such as the backtrace. 

In the same way, `BXIEXIT()` ensures all logs are flushed before actually exiting and provides more information.

#### Fork Support

Of course, the `bxilog` module is thread-safe (and lock-free), but it is also *fork-safe*. It basically,  means that it supports the standard POSIX `fork()` system call that creates a new process. In the face of a multi-threaded program which is always the case with the `bxilog` module, this is very hard to implement and usually unsupported. If you don't believe this statement, have a look to the RATIONALE section in the POSIX documentation of the [`fork()` system call](http://pubs.opengroup.org/onlinepubs/009695399/functions/fork.html) and also to the following articles:

* http://www.linuxprogrammingblog.com/threads-and-fork-think-twice-before-using-them
* http://thorstenball.com/blog/2014/10/13/why-threads-cant-fork/

When forking a child, the `bxilog` configuration remains the same in the parent, and the module is still configured as before the `fork()` call. However, in the child, this is not the case: the `bxilog` module has not been initialized. This is somewhat in opposition to the semantic of the `fork()` system call which is supposed to clone the entire process, and thus, to produce an exact equivalent of the parent. However, if you have seen the POSIX documentation, in the case of multi-threaded programs, this is no longer the case as only one thread is cloned: the one calling `fork()`. Anyway, the main reason is to prevent the overhead of a logging initialization with all the handlers if the only purpose of the `fork()` is the call to `execve()` which actually overwrite the whole process with a new program.

So, when using `bxilog`, after a `fork()`, keep in mind the following rules:

* the parent can continue to log as usual;
* the child can do an `execve` as usual with no overhead, the `bxilog` has not been initialized;
* the child might want to produce log itself, in this case, it can configure the `bxilog` as usual.

Note that with such a mechanism, the child can either use the same configuration as its parent, or use a distinct configuration. 

Consider the following example:

In [21]:
#include <unistd.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <errno.h>
#include <sysexits.h>

#include <bxi/base/err.h>
#include <bxi/base/time.h>
#include <bxi/base/log.h>
#include <bxi/base/log/console_handler.h>
#include <bxi/base/log/file_handler.h>

SET_LOGGER(PARENT_LOGGER, "parent");
SET_LOGGER(CHILD_LOGGER, "child");

int main() {
    bxilog_config_p config = bxilog_config_new("Fork Example");
    bxilog_config_add_handler(config, 
                              BXILOG_CONSOLE_HANDLER, 
                              BXILOG_FILTERS_ALL_ALL, 
                              BXILOG_WARNING,
                              12, // Use 12 characters max when displaying logger names
                              BXILOG_COLORS_NONE);
    
    bxierr_p err = bxilog_init(config);
    bxierr_abort_ifko(err);
    
    INFO(PARENT_LOGGER, "In the parent process");
    
    size_t loop_nb = 5; 
    errno = 0;
    int rc = fork();
    if (-1 == rc) {
        err = bxierr_errno("Calling fork() failed");
        BXIEXIT(EX_OSERR, err, PARENT_LOGGER, BXILOG_CRITICAL);
    }
    if (0 == rc) {  // In the child
        // Configure the logging system: we use the same config
        bxierr_p err = bxilog_init(config);
        bxierr_abort_ifko(err);
    
        INFO(CHILD_LOGGER, "In the child process");
        for (size_t i = 0; i < loop_nb; i++) {
            INFO(CHILD_LOGGER, "Logging something at step %zu", i);
            bxierr_p err = bxitime_sleep(CLOCK_MONOTONIC, 0, 4e8);
            BXILOG_REPORT(CHILD_LOGGER, BXILOG_NOTICE, err, "Continuing...");
        }
        
    } else { // In the parent
        for (size_t i = 0; i < loop_nb; i++) {
            INFO(PARENT_LOGGER, "Logging something at step %zu", i);
            bxierr_p err = bxitime_sleep(CLOCK_MONOTONIC, 0, 4e8);
            BXILOG_REPORT(PARENT_LOGGER, BXILOG_NOTICE, err, "Continuing...");
        }
    }
    
    err = bxilog_finalize(true);
    bxierr_abort_ifko(err);
    
}


[D] ~bxilog      Initialization done
[I] parent       In the parent process
[F] ~bxilog.fork Preparing for a fork() (state == 3)
[F] ~bxilog      Requesting a flush()
[F] ~bxilog      flush() done succesfully on all 1 handlers
[F] ~bxilog.fork Ready after a fork()
[I] parent       Logging something at step 0
[I] parent       Logging something at step 1
[I] parent       Logging something at step 2
[D] ~bxilog      Initialization done
[I] child        In the child process
[I] child        Logging something at step 0
[I] child        Logging something at step 1
[I] child        Logging something at step 2
[I] parent       Logging something at step 3
[I] parent       Logging something at step 4
[D] ~bxilog      Exiting bxilog
[I] child        Logging something at step 3
[I] child        Logging something at step 4
[D] ~bxilog      Exiting bxilog


Note that logs won't mix-up between child and parents even if they share the same configuration.


### Signal Catching

The signaling system is a powerfull but very hard to use mechanism: asynchronous and synchronous signals must be distinguished, some signals are targetting a specific thread (SIGSEGV, SIGFPE) while others are targetting the process as a whole (SIGTERM, SIGQUIT, or when `kill()` has been used). In this last case, any thread that is not blocking the signal can be interrupted (in the middle of any function...) in order to process the signal. By default, the behavior is to just terminate the whole process. The end result, is often a useless message as shown by the example below:

In [22]:
#include <bxi/base/err.h>
#include <bxi/base/log.h>
#include <bxi/base/log/console_handler.h>

SET_LOGGER(MAIN_LOGGER, "main");

int main(int argc, char ** argv) {
    bxilog_config_p config = bxilog_config_new("Signal Example");
    bxilog_config_add_handler(config, 
                              BXILOG_CONSOLE_HANDLER, 
                              BXILOG_FILTERS_ALL_ALL, 
                              BXILOG_WARNING,
                              12, // Use 12 characters max when displaying logger names
                              BXILOG_COLORS_NONE);
    
    bxierr_p err = bxilog_init(config);
    bxierr_abort_ifko(err);
    
    OUT(MAIN_LOGGER, "%s", argv[0]);
    OUT(MAIN_LOGGER, "%s", (char*) 0x1); 
    //printf("%s\n", argv[0]);
    //printf("%s\n", (char*) 0x1);
}

[C kernel] Executable exited with code -11

In this simple example, we expect the signal SIGSEGV (code = -11) to be raised since we access the memory at address `0x1`. However what is not expected is the lost of messages: at least `argv[0]`, that is the name of the program should have been written to the standard output. Actually, this has nothing to do with the `bxilog` module: replace the `OUT()` instruction by a `printf`, the result will be the same. The function is called, but due to some buffering, the output is not seen immediately on the screen. And the signal `SIGSEGV` is raised before the flush of this buffer happened. 

For most simple C programs, the `bxilog` module provides the `bxilog_install_handler()` function that tries its best to flush all buffers in all log handlers. Furthermore, it also provides much more information than the simple default system as shown by the example below:  

In [23]:
#include <bxi/base/err.h>
#include <bxi/base/log.h>
#include <bxi/base/log/console_handler.h>

SET_LOGGER(MAIN_LOGGER, "main");

int main(int argc, char ** argv) {
    bxilog_config_p config = bxilog_config_new("Signal Example");
    bxilog_config_add_handler(config, 
                              BXILOG_CONSOLE_HANDLER, 
                              BXILOG_FILTERS_ALL_ALL, 
                              BXILOG_WARNING,
                              12, // Use 12 characters max when displaying logger names
                              BXILOG_COLORS_NONE);
    
    bxierr_p err = bxilog_init(config);
    bxierr_abort_ifko(err);
    
    // Installing signal handler
    err = bxilog_install_sighandler();
    
    BXILOG_REPORT(MAIN_LOGGER, BXILOG_ERROR, err, "Can't install signal handlers, continuing");
    OUT(MAIN_LOGGER, "%s", argv[0]);
    OUT(MAIN_LOGGER, "%s", (char*) 0x1); 
    
}

[C kernel] Executable exited with code -11Signal Example: Signal=11 ('Segmentation fault'), Signal Code=1 (man 2 sigaction)
##trce## Backtrace of tid 173: 10 function calls 
##trce## [00] /lib64/libbxibase.so.0(bxierr_backtrace_str+0x195) [0x7ff9655ddf15]
##trce## [01] /lib64/libbxibase.so.0(+0x14dc1) [0x7ff9655e9dc1]
##trce## [02] /lib64/libc.so.6(+0x35250) [0x7ff965249250]
##trce## [03] /lib64/libc.so.6(_IO_vfprintf+0x1564) [0x7ff96525c694]
##trce## [04] /lib64/libc.so.6(__vsnprintf_chk+0x95) [0x7ff9653208d5]
##trce## [05] /lib64/libbxibase.so.0(bxilog_logger_vlog_nolevelcheck+0x14f) [0x7ff9655e63cf]
##trce## [06] /lib64/libbxibase.so.0(bxilog_logger_log_nolevelcheck+0xa5) [0x7ff9655e65c5]
##trce## [07] /tmp/tmplms298cf.out() [0x400cfe]
##trce## [08] /lib64/libc.so.6(__libc_start_main+0xf5) [0x7ff965235b35]
##trce## [09] /tmp/tmplms298cf.out() [0x400a79]
##trce## Backtrace end

[C] ~bxilog.sign Signal Example: Signal=11 ('Segmentation fault'), Signal Code=1 (man 2 sigaction)
[C] ~bxi

[D] ~bxilog      Initialization done
[F] ~bxilog.sign Alternate signal stack set at 0x619cc0 (8192 B)
[D] ~bxilog.sign Setting signal handler process wide
[D] ~bxilog.sign Signal handler set for 11: Segmentation fault
[D] ~bxilog.sign Signal handler set for 7: Bus error
[D] ~bxilog.sign Signal handler set for 8: Floating point exception
[D] ~bxilog.sign Signal handler set for 4: Illegal instruction
[D] ~bxilog.sign Signal handler set for 2: Interrupt
[D] ~bxilog.sign Signal handler set for 15: Terminated
[I] ~bxilog.sign Signal handlers set
/tmp/tmplms298cf.out
[F] ~bxilog      Requesting a flush()
[F] ~bxilog      flush() done succesfully on all 1 handlers


Not only, all logs have been flushed, but we also have various information on the signal, and the stacktrace. Here we have twice the output because on signal, the signal handler also displays signaling information on the standard error. 


### Performance

The `bxilog` module is an industrially designed logging API with very good performance. It has:

* low overhead: a thread that produces a log does not perform any I/O operation; the 
  minimum amount of time is spent producing the log, the system take care of the transport to 
  its destination (console, filesystem, network, ...).     
* Low latency and high throughput: the design is thread-safe but lock-less using the `bxizmq` module 
  for its inter-thread communication;
  
[TODO: present the bench result]